<a href="https://colab.research.google.com/github/chienyuanhung/twitter-sentimometer/blob/master/nlp_deep_learning_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
import io
import re
import json
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [73]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving training.1600000.processed.noemoticon.csv to training.1600000.processed.noemoticon (1).csv


KeyboardInterrupt: ignored

In [74]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['training.1600000.processed.noemoticon.csv']), encoding='latin-1', header = None)

df2.head()


,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [75]:
# take the target and tweet for training, create a new Data Frame

df1 = df2[[0, 5]]
df1.columns = ['target', 'tweet']
df1.head()


,target,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [0]:
# function for getting rid of unwanted characters in tweets

def getCleanTwit(str1):
    words= str1.split()
    clean_words = []
    for i in words:
        if re.findall(r'^http', i):
            continue
        else:
            clean_words.append(i)
    text = u' '.join(x for x in clean_words)
    text = re.sub("http*:\/\/[0-9a-zA-Z\-\_]+\.[\-\_0-9a-zA-Z]", u"", text)
    text = re.sub("\@[\_0-9a-zA-Z]+\:?", u"", text)
    text = re.sub("\&amp\;?", u"", text)
    text = re.sub("[\:\.]{1,}$", u"", text)
    text = re.sub("^RT\:?", u"", text)

    return text

In [0]:
tweets = df1.tweet.to_list()
# the list for clean tweet
all_tweets = []
for i in tweets:
    twi = getCleanTwit(i)
    all_tweets.append(twi)
    

In [78]:
len(all_tweets)

1600000

In [0]:
# change the label from 0 and 4 to 0.0 and 1.0

label = df1['target'].to_list()
label_1 = []
for i in label:
  if i == 0:
    label_1.append(0.0)
  elif i == 4:
    label_1.append(1.0)

In [0]:
labels = label_1[0: 400000] + label_1[800000:1200000]

features = all_tweets[0: 400000] + all_tweets[800000:1200000]


In [0]:
# split training and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=41)

In [0]:
# tokenize the training and test set
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
data_text = X_train + X_test
tokenizer.fit_on_texts(data_text)

In [0]:
if num_words is None:
    num_words = len(tokenizer.word_index)
    

In [0]:
x_train_tokens = tokenizer.texts_to_sequences(X_train)
x_test_tokens = tokenizer.texts_to_sequences(X_test)

In [0]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
# max_tokens= np.max(num_tokens)
max_tokens = 280

In [0]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [0]:
from keras.models import Sequential
from keras import layers




In [88]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(num_words, embedding_dim, input_length=max_tokens))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 280, 100)          1000000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 276, 128)          64128     
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 1,065,429
Trainable params: 1,065,429
Non-trainable params: 0
_________________________________________________________________


In [89]:
model.fit(x_train_pad, y_train, validation_data=(x_test_pad, y_test), epochs=2, batch_size=64)

Train on 640000 samples, validate on 160000 samples
Epoch 1/2
640000/640000 [==============================] - 1983s 3ms/step - loss: 0.4270 - acc: 0.8016 - val_loss: 0.4048 - val_acc: 0.8151
Epoch 2/2
640000/640000 [==============================] - 1979s 3ms/step - loss: 0.3746 - acc: 0.8324 - val_loss: 0.4052 - val_acc: 0.8165


In [90]:
%%time
result = model.evaluate(x_test_pad, y_test)

160000/160000 [==============================] - 119s 743us/step
CPU times: user 3min 29s, sys: 2.93 s, total: 3min 32s
Wall time: 1min 58s


In [91]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 81.65%


In [0]:
from google.colab import files

model.save("cnn_model.h5")

In [0]:
files.download('cnn_model.h5')

Training with stopwords removed

In [0]:
import nltk
nltk.download('stopwords')

In [1]:
# import the library to tokenize and remove stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

# remove the stopwords from twees
stopwords_removed = []

for i in all_tweets:
    text = []
    for j in i.split():
        j = j.lower()
        j = j.replace('.', '')
        j = j.replace(',', '')
        j = j.replace('?', '')
        if j not in stopWords:
            text.append(j)
            
    stopwords_removed.append(u' '.join(x for x in text))


LookupError: ignored